# Data Load

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"peddireddigari","key":"7dcabc8dee9ca1cac4fb100249dc718b"}'}

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c dogs-vs-cats

100% 810M/812M [00:21<00:00, 43.6MB/s]
100% 812M/812M [00:21<00:00, 39.1MB/s]


In [4]:
!unzip -qq dogs-vs-cats.zip

In [5]:
!unzip -qq train.zip

In [6]:
#Google colab tensor flow version is 2.13. So, older version is installed
!pip install tensorflow==2.12


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1

In [7]:
import tensorflow as tf
tf.__version__         # '2.12.0'

'2.12.0'

In [8]:
import numpy as np

# Set the seed for NumPy's random number generator
np.random.seed(42)  # You can use any integer value as the seed

# Question 1

In [9]:
#creating the directories and assigning the subset of the cat and dog images for the traing validation and testing.
import os, shutil, pathlib

#shutil.rmtree("cats_vs_dogs_small")

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)


In [10]:
#Building the Model.
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
#Running the summary of the model.
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                         

In [11]:
#Compiling the model.
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [12]:
#Creating image datasets from directories for training, validation, and testing, each with specified image size and batch size.
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [13]:
## Using the callback function to save the model checkpoint with the best validation loss and also training the model while monitoring the validation loss
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)


Epoch 1/30
63/63 [==============================] - 17s 88ms/step - loss: 0.7119 - accuracy: 0.4935 - val_loss: 0.6920 - val_accuracy: 0.6030
Epoch 2/30
63/63 [==============================] - 4s 60ms/step - loss: 0.6924 - accuracy: 0.5405 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 3/30
63/63 [==============================] - 5s 82ms/step - loss: 0.6974 - accuracy: 0.5255 - val_loss: 0.6873 - val_accuracy: 0.5300
Epoch 4/30
63/63 [==============================] - 6s 86ms/step - loss: 0.6847 - accuracy: 0.5690 - val_loss: 0.8924 - val_accuracy: 0.5000
Epoch 5/30
63/63 [==============================] - 5s 82ms/step - loss: 0.6612 - accuracy: 0.6290 - val_loss: 0.6430 - val_accuracy: 0.6180
Epoch 6/30
63/63 [==============================] - 5s 80ms/step - loss: 0.6240 - accuracy: 0.6575 - val_loss: 0.6186 - val_accuracy: 0.6370
Epoch 7/30
63/63 [==============================] - 6s 82ms/step - loss: 0.5866 - accuracy: 0.6985 - val_loss: 0.5920 - val_accuracy: 0.6650
Epoch 8/30
6

In [14]:
#Testing the model accuracy.
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 4s 59ms/step - loss: 0.5880 - accuracy: 0.6930
Test accuracy: 0.693


Using the Data_augumentation

In [15]:
#Defining the data_augumentation stage.
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [16]:
#Building the model and compiling it.
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [17]:
#Using the callback function to save the model checkpoint with the best validation loss and also training the model while monitoring the validation loss
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/100
63/63 [==============================] - 6s 63ms/step - loss: 0.7080 - accuracy: 0.5080 - val_loss: 0.6943 - val_accuracy: 0.5000
Epoch 2/100
63/63 [==============================] - 5s 76ms/step - loss: 0.6953 - accuracy: 0.5165 - val_loss: 0.6914 - val_accuracy: 0.5080
Epoch 3/100
63/63 [==============================] - 6s 91ms/step - loss: 0.6898 - accuracy: 0.5500 - val_loss: 0.6913 - val_accuracy: 0.5030
Epoch 4/100
63/63 [==============================] - 4s 63ms/step - loss: 0.6859 - accuracy: 0.5560 - val_loss: 0.6831 - val_accuracy: 0.5500
Epoch 5/100
63/63 [==============================] - 4s 61ms/step - loss: 0.6669 - accuracy: 0.6095 - val_loss: 0.6550 - val_accuracy: 0.6150
Epoch 6/100
63/63 [==============================] - 8s 114ms/step - loss: 0.6454 - accuracy: 0.6385 - val_loss: 0.6591 - val_accuracy: 0.5970
Epoch 7/100
63/63 [==============================] - 4s 63ms/step - loss: 0.6398 - accuracy: 0.6440 - val_loss: 0.6317 - val_accuracy: 0.6270
Epoch

In [18]:
#Testing the model accuracy.
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 3s 36ms/step - loss: 0.4465 - accuracy: 0.8095
Test accuracy: 0.809


# Question 2

In [20]:
#creating the directories and assigning the subset of the cat and dog images for the traing validation and testing.
#Increasing the training samples from 1000 to 2000
import os, shutil, pathlib

#shutil.rmtree("cats_vs_dogs_Increased_trainsamples")

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_Increased_trainsamples")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=2000)
make_subset("validation", start_index=2000, end_index=2500)
make_subset("test", start_index=2500, end_index=3500)

In [21]:
#Building the model and compiling it.
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 89, 89, 32)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                       

In [22]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [23]:
#Using the callback function to save the model checkpoint with the best validation loss and also training the model while monitoring the validation loss
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)


Epoch 1/30
63/63 [==============================] - 9s 114ms/step - loss: 0.7199 - accuracy: 0.5415 - val_loss: 0.7031 - val_accuracy: 0.5000
Epoch 2/30
63/63 [==============================] - 4s 59ms/step - loss: 0.6939 - accuracy: 0.5235 - val_loss: 0.6820 - val_accuracy: 0.5190
Epoch 3/30
63/63 [==============================] - 4s 59ms/step - loss: 0.6823 - accuracy: 0.5805 - val_loss: 0.6884 - val_accuracy: 0.5650
Epoch 4/30
63/63 [==============================] - 7s 113ms/step - loss: 0.6323 - accuracy: 0.6430 - val_loss: 0.6273 - val_accuracy: 0.6450
Epoch 5/30
63/63 [==============================] - 4s 60ms/step - loss: 0.6006 - accuracy: 0.6900 - val_loss: 0.5956 - val_accuracy: 0.6750
Epoch 6/30
63/63 [==============================] - 4s 59ms/step - loss: 0.6026 - accuracy: 0.7000 - val_loss: 0.6597 - val_accuracy: 0.6420
Epoch 7/30
63/63 [==============================] - 6s 88ms/step - loss: 0.5210 - accuracy: 0.7470 - val_loss: 0.6454 - val_accuracy: 0.6470
Epoch 8/30


In [24]:
#Testing the model accuracy.
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 5s 64ms/step - loss: 0.5933 - accuracy: 0.6950
Test accuracy: 0.695


# Question 3

##3000 Sample Size

In [26]:
#Increasing the training sample size to find the optimal training sample size.
import os, shutil, pathlib

#shutil.rmtree("cats_vs_dogs_Increased_trainsamples")

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_Increased_trainsamples")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=3000)
make_subset("validation", start_index=3000, end_index=3500)
make_subset("test", start_index=3500, end_index=4500)


In [27]:
#Building the model and compiling it.
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling_3 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 89, 89, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 43, 43, 64)       0         
 g2D)                                                      

In [28]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [29]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)


Epoch 1/30
63/63 [==============================] - 9s 108ms/step - loss: 0.7052 - accuracy: 0.5045 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 2/30
63/63 [==============================] - 4s 61ms/step - loss: 0.6989 - accuracy: 0.5130 - val_loss: 0.6882 - val_accuracy: 0.5740
Epoch 3/30
63/63 [==============================] - 4s 60ms/step - loss: 0.6897 - accuracy: 0.5505 - val_loss: 0.6735 - val_accuracy: 0.6170
Epoch 4/30
63/63 [==============================] - 6s 90ms/step - loss: 0.6679 - accuracy: 0.5915 - val_loss: 0.6423 - val_accuracy: 0.6310
Epoch 5/30
63/63 [==============================] - 5s 74ms/step - loss: 0.6395 - accuracy: 0.6425 - val_loss: 0.6394 - val_accuracy: 0.6310
Epoch 6/30
63/63 [==============================] - 4s 59ms/step - loss: 0.6043 - accuracy: 0.6790 - val_loss: 0.6223 - val_accuracy: 0.6590
Epoch 7/30
63/63 [==============================] - 4s 66ms/step - loss: 0.5805 - accuracy: 0.6925 - val_loss: 0.5798 - val_accuracy: 0.6950
Epoch 8/30
6

In [30]:
#Testing the model accuracy.
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 3s 35ms/step - loss: 0.5796 - accuracy: 0.6990
Test accuracy: 0.699


##4000 Sample Size

In [31]:
#Increasing the training sample size to find the optimal training sample size.
import os, shutil, pathlib

#shutil.rmtree("cats_vs_dogs_Increasedto4000_trainsamples")

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_Increasedto4000_trainsamples")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=4000)
make_subset("validation", start_index=4000, end_index=4500)
make_subset("test", start_index=4500, end_index=5500)


In [32]:
#Building the model and compiling it.
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling_4 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 89, 89, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 43, 43, 64)       0         
 g2D)                                                      

In [33]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [34]:
#Using the callback function to save the model checkpoint with the best validation loss and also training the model while monitoring the validation loss
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch4000.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 7s 60ms/step - loss: 0.7394 - accuracy: 0.4955 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 2/30
63/63 [==============================] - 4s 57ms/step - loss: 0.7149 - accuracy: 0.5135 - val_loss: 0.6930 - val_accuracy: 0.5010
Epoch 3/30
63/63 [==============================] - 7s 104ms/step - loss: 0.6936 - accuracy: 0.5225 - val_loss: 0.6984 - val_accuracy: 0.5000
Epoch 4/30
63/63 [==============================] - 4s 57ms/step - loss: 0.7034 - accuracy: 0.5340 - val_loss: 0.6868 - val_accuracy: 0.5230
Epoch 5/30
63/63 [==============================] - 4s 67ms/step - loss: 0.6877 - accuracy: 0.5715 - val_loss: 0.6755 - val_accuracy: 0.5860
Epoch 6/30
63/63 [==============================] - 6s 90ms/step - loss: 0.6627 - accuracy: 0.6160 - val_loss: 0.6523 - val_accuracy: 0.6210
Epoch 7/30
63/63 [==============================] - 4s 59ms/step - loss: 0.6103 - accuracy: 0.6635 - val_loss: 0.5850 - val_accuracy: 0.6780
Epoch 8/30
6

In [35]:
#Testing the model accuracy.
test_model = keras.models.load_model(
    "convnet_from_scratch4000.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 3s 34ms/step - loss: 0.5672 - accuracy: 0.7335
Test accuracy: 0.734


# Question 4:

## Leveraging Pretrained Network Model

In [36]:
#creating the directories and assigning the subset of the cat and dog images for the traing validation and testing.
import os
import shutil
import pathlib
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_Pretrained")

#shutil.rmtree("cats_vs_dogs_small_Pretrained")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [37]:
# Loading  the pretrained weights to VGG16 model.
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))
conv_base.summary()

58889256/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                              

In [38]:
#Extracting the VGG16 features and their corresponding labels.
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)


1/1 [==============================] - 0s 22ms/step


In [39]:
train_features.shape
test_features.shape

(2000, 5, 5, 512)

In [40]:
#Building the model.
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction.keras",
      save_best_only=True,
      monitor="val_loss")
]

history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

Epoch 1/20
63/63 [==============================] - 2s 15ms/step - loss: 19.6610 - accuracy: 0.9250 - val_loss: 2.9619 - val_accuracy: 0.9680
Epoch 2/20
63/63 [==============================] - 0s 6ms/step - loss: 3.3102 - accuracy: 0.9740 - val_loss: 4.2211 - val_accuracy: 0.9750
Epoch 3/20
63/63 [==============================] - 0s 7ms/step - loss: 3.0770 - accuracy: 0.9815 - val_loss: 3.2062 - val_accuracy: 0.9790
Epoch 4/20
63/63 [==============================] - 0s 7ms/step - loss: 1.8333 - accuracy: 0.9885 - val_loss: 2.3196 - val_accuracy: 0.9800
Epoch 5/20
63/63 [==============================] - 0s 7ms/step - loss: 0.3846 - accuracy: 0.9955 - val_loss: 2.7487 - val_accuracy: 0.9740
Epoch 6/20
63/63 [==============================] - 0s 6ms/step - loss: 0.4341 - accuracy: 0.9940 - val_loss: 3.9593 - val_accuracy: 0.9760
Epoch 7/20
63/63 [==============================] - 0s 7ms/step - loss: 1.0141 - accuracy: 0.9945 - val_loss: 4.5357 - val_accuracy: 0.9750
Epoch 8/20
63/63 [

In [41]:
#Testing the model
test_model = keras.models.load_model("feature_extraction.keras")
test_loss, test_acc = test_model.evaluate(test_features, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 0s 3ms/step - loss: 5.2698 - accuracy: 0.9695
Test accuracy: 0.970


In [42]:
#Loading pre-trained weights to the VGG16 model
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False
# Freezing the layers of the pretrained CNN and printing the list of trainable weights.
conv_base.trainable = True
print("This is the number of trainable weights "
      "before freezing the conv base:", len(conv_base.trainable_weights))
#UnFreezing the layers of the pretrained CNN and printing the list of trainable weights.
conv_base.trainable = False
print("This is the number of trainable weights "
      "after freezing the conv base:", len(conv_base.trainable_weights))


This is the number of trainable weights before freezing the conv base: 26
This is the number of trainable weights after freezing the conv base: 0


## Model with Data Augmentation

In [43]:
#Adding a data augumentation stage.
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [44]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
    ]

In [45]:

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/50
63/63 [==============================] - 9s 96ms/step - loss: 15.7015 - accuracy: 0.9020 - val_loss: 6.2067 - val_accuracy: 0.9590
Epoch 2/50
63/63 [==============================] - 5s 71ms/step - loss: 6.3461 - accuracy: 0.9485 - val_loss: 4.2438 - val_accuracy: 0.9720
Epoch 3/50
63/63 [==============================] - 7s 107ms/step - loss: 6.9581 - accuracy: 0.9490 - val_loss: 6.8369 - val_accuracy: 0.9640
Epoch 4/50
63/63 [==============================] - 5s 68ms/step - loss: 5.5540 - accuracy: 0.9590 - val_loss: 3.7349 - val_accuracy: 0.9750
Epoch 5/50
63/63 [==============================] - 6s 89ms/step - loss: 4.1061 - accuracy: 0.9680 - val_loss: 2.6019 - val_accuracy: 0.9780
Epoch 6/50
63/63 [==============================] - 4s 65ms/step - loss: 3.6227 - accuracy: 0.9700 - val_loss: 3.3272 - val_accuracy: 0.9750
Epoch 7/50
63/63 [==============================] - 5s 75ms/step - loss: 3.0576 - accuracy: 0.9715 - val_loss: 3.6197 - val_accuracy: 0.9760
Epoch 8/50


In [46]:
#Testing the model accuracy.
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 3s 41ms/step - loss: 2.1288 - accuracy: 0.9760
Test accuracy: 0.976


##2500 Sample Size

In [47]:
#Increasing the pretrained training sample.
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_PretrainedIncreased_trainsample")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir,exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)
make_subset("train", start_index=0, end_index=2500)
make_subset("validation", start_index=1500, end_index=2000)
make_subset("test", start_index=2000, end_index=3000)

In [48]:
#Loading pre-trained weights to the VGG16 model
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))
conv_base.trainable = False

In [49]:
#Building the model.
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction8.keras",
      save_best_only=True,
      monitor="val_loss")
]
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

Epoch 1/20
63/63 [==============================] - 1s 10ms/step - loss: 20.8962 - accuracy: 0.9240 - val_loss: 4.8184 - val_accuracy: 0.9710
Epoch 2/20
63/63 [==============================] - 0s 7ms/step - loss: 3.0084 - accuracy: 0.9755 - val_loss: 8.4755 - val_accuracy: 0.9570
Epoch 3/20
63/63 [==============================] - 1s 8ms/step - loss: 2.6969 - accuracy: 0.9820 - val_loss: 7.5511 - val_accuracy: 0.9570
Epoch 4/20
63/63 [==============================] - 1s 10ms/step - loss: 1.4646 - accuracy: 0.9905 - val_loss: 4.1693 - val_accuracy: 0.9770
Epoch 5/20
63/63 [==============================] - 1s 9ms/step - loss: 1.2250 - accuracy: 0.9925 - val_loss: 4.9747 - val_accuracy: 0.9790
Epoch 6/20
63/63 [==============================] - 1s 8ms/step - loss: 0.7259 - accuracy: 0.9940 - val_loss: 4.1945 - val_accuracy: 0.9770
Epoch 7/20
63/63 [==============================] - 1s 10ms/step - loss: 0.6651 - accuracy: 0.9955 - val_loss: 4.1028 - val_accuracy: 0.9790
Epoch 8/20
63/63

In [50]:
#Testing the accuracy of teh model.
test_model = keras.models.load_model("feature_extraction8.keras")

test_loss, test1_acc = test_model.evaluate(test_features, test_labels)
print(f"Test accuracy: {test1_acc:.3f}")

63/63 [==============================] - 0s 3ms/step - loss: 5.7532 - accuracy: 0.9735
Test accuracy: 0.974


##3000 Sample Size

In [51]:
#creating the directories and assigning the subset of the cat and dog images for the traing validation and testing.
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_Pretrained_optimal_trainsample")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir,exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)
make_subset("train", start_index=0, end_index=3000)
make_subset("validation", start_index=3000, end_index=3500)
make_subset("test", start_index=3500, end_index=4500)


In [52]:
#Loading pre-trained weights to the VGG16 model
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False

In [53]:
#Building the model.
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction1.keras",
      save_best_only=True,
      monitor="val_loss")
]
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

Epoch 1/20
63/63 [==============================] - 3s 15ms/step - loss: 21.4749 - accuracy: 0.9235 - val_loss: 4.1144 - val_accuracy: 0.9670
Epoch 2/20
63/63 [==============================] - 0s 6ms/step - loss: 3.0298 - accuracy: 0.9715 - val_loss: 4.3684 - val_accuracy: 0.9670
Epoch 3/20
63/63 [==============================] - 0s 8ms/step - loss: 1.8989 - accuracy: 0.9820 - val_loss: 3.9378 - val_accuracy: 0.9750
Epoch 4/20
63/63 [==============================] - 0s 6ms/step - loss: 1.3599 - accuracy: 0.9880 - val_loss: 6.2443 - val_accuracy: 0.9670
Epoch 5/20
63/63 [==============================] - 0s 7ms/step - loss: 0.9227 - accuracy: 0.9920 - val_loss: 5.8897 - val_accuracy: 0.9750
Epoch 6/20
63/63 [==============================] - 0s 7ms/step - loss: 0.8812 - accuracy: 0.9940 - val_loss: 3.8164 - val_accuracy: 0.9750
Epoch 7/20
63/63 [==============================] - 0s 6ms/step - loss: 0.6475 - accuracy: 0.9920 - val_loss: 4.4045 - val_accuracy: 0.9790
Epoch 8/20
63/63 [

In [54]:
#Testing the accuracy of teh model.
test_model = keras.models.load_model(
    "feature_extraction1.keras")
test_loss, test1_acc = test_model.evaluate(test_features, test_labels)
print(f"Test accuracy: {test1_acc:.3f}")

63/63 [==============================] - 0s 3ms/step - loss: 5.9356 - accuracy: 0.9750
Test accuracy: 0.975


##4000 Sample Size

In [55]:
#creating the directories and assigning the subset of the cat and dog images for the traing validation and testing.
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_Pretrained_optimal_trainsample")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir,exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)
make_subset("train", start_index=0, end_index=4000)
make_subset("validation", start_index=4000, end_index=4500)
make_subset("test", start_index=4500, end_index=5500)


In [56]:
#Loading pre-trained weights to the VGG16 model
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False

In [57]:
#Building the model.
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction2.keras",
      save_best_only=True,
      monitor="val_loss")
]
history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

Epoch 1/20
63/63 [==============================] - 1s 10ms/step - loss: 18.8746 - accuracy: 0.9215 - val_loss: 6.0421 - val_accuracy: 0.9650
Epoch 2/20
63/63 [==============================] - 0s 7ms/step - loss: 4.0287 - accuracy: 0.9750 - val_loss: 3.9182 - val_accuracy: 0.9700
Epoch 3/20
63/63 [==============================] - 0s 6ms/step - loss: 1.9595 - accuracy: 0.9825 - val_loss: 5.6370 - val_accuracy: 0.9730
Epoch 4/20
63/63 [==============================] - 0s 5ms/step - loss: 1.2757 - accuracy: 0.9920 - val_loss: 5.0182 - val_accuracy: 0.9680
Epoch 5/20
63/63 [==============================] - 0s 6ms/step - loss: 1.1497 - accuracy: 0.9905 - val_loss: 4.3385 - val_accuracy: 0.9740
Epoch 6/20
63/63 [==============================] - 0s 7ms/step - loss: 0.7530 - accuracy: 0.9930 - val_loss: 7.7387 - val_accuracy: 0.9640
Epoch 7/20
63/63 [==============================] - 0s 7ms/step - loss: 0.8058 - accuracy: 0.9955 - val_loss: 4.8581 - val_accuracy: 0.9740
Epoch 8/20
63/63 [

In [58]:
#Testing the accuracy of the model.
#Testing the accuracy of the model.
test_model = keras.models.load_model(
    "feature_extraction2.keras")
test_loss, test1_acc = test_model.evaluate(test_features, test_labels)
print(f"Test accuracy: {test1_acc:.3f}")

63/63 [==============================] - 0s 4ms/step - loss: 4.8016 - accuracy: 0.9710
Test accuracy: 0.971
